### From articles "Options for smart investor"

https://dzen.ru/id/5f6b2336d03ed90da6bf9d00


In [4]:
import datetime
import pandas as pd
from chart_studio import plotly

from plotly import graph_objs as go
from plotly.offline import iplot, init_notebook_mode
#import plotly.express as px

from option_lib.provider import PandasLocalFileProvider, RequestParameters
from option_lib.entities import OptionType, OptionPriceStatus
from option_lib import Option
from option_lib.entities import LegType, OptionLeg


init_notebook_mode(connected=True)

pd.set_option("display.max_rows", 20, "display.max_columns", 30)

In [9]:
cme_provider = PandasLocalFileProvider('CME', '../data')
cur_dt = datetime.date.today()
provider_params = RequestParameters(period_to=cur_dt.year-1)
brn = Option(cme_provider, 'BRN', provider_params, option_columns=['datetime', 'expiration_date','strike', 'type', 'premium', 'iv',  'futures_expiration_date', 'delta', 'gamma', 'vega', 'theta', 'quick_delta', 'contract_size'])

In [10]:
brn.df_hist.head(2)

,datetime,expiration_date,strike,type,premium,iv,futures_expiration_date,delta,gamma,vega,theta,quick_delta,contract_size
0,2024-12-23,2024-12-23,6.0,c,66.629997,4.999922,2024-12-30,1.0,0.0,0.0,NaN,1.0,1000.0
1,2024-12-23,2024-12-23,6.0,p,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,-0.0,1000.0


In [11]:
df_fut = brn.df_fut
df_fut.head(2)

,datetime,expiration_date,price
0,2024-12-23,2024-12-30,72.63
1,2024-12-23,2025-01-31,72.32


In [12]:
brn.enrichment.add_future().add_intrinsic_and_time_value()
brn.df_hist.head(2)

,datetime,expiration_date,strike,type,premium,iv,futures_expiration_date,delta,gamma,vega,theta,quick_delta,contract_size,futures_price,intrinsic_value,time_value
0,2024-12-23,2024-12-23,6.0,c,66.629997,4.999922,2024-12-30,1.0,0.0,0.0,NaN,1.0,1000.0,72.63,66.63,-0.000003
1,2024-12-23,2024-12-23,6.0,p,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,-0.0,1000.0,72.63,0.00,0.010000


In [13]:
brn.enrichment.add_atm_itm_otm()
brn.df_hist[brn.df_hist['price_status']==OptionPriceStatus.ATM.code].head(2)

,datetime,expiration_date,strike,type,premium,iv,futures_expiration_date,delta,gamma,vega,theta,quick_delta,contract_size,futures_price,intrinsic_value,time_value,price_status
144,2024-12-23,2024-12-23,72.75,c,0.53,4.999922,2024-12-30,0.0,0.0,0.0,NaN,0.0,1000.0,72.63,0.00,5.300000e-01,atm
145,2024-12-23,2024-12-23,72.75,p,0.12,4.999922,2024-12-30,-1.0,0.0,0.0,NaN,-1.0,1000.0,72.63,0.12,-2.682214e-09,atm


In [14]:
df_opt = brn.df_hist
df_opt

,datetime,expiration_date,strike,type,premium,iv,futures_expiration_date,delta,gamma,vega,theta,quick_delta,contract_size,futures_price,intrinsic_value,time_value,price_status
0,2024-12-23,2024-12-23,6.0,c,66.629997,4.999922,2024-12-30,1.000000,0.000000,0.000000,NaN,1.000000,1000.0,72.63,66.63,-0.000003,itm
1,2024-12-23,2024-12-23,6.0,p,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,-0.000000,1000.0,72.63,0.00,0.010000,otm
2,2024-12-23,2024-12-23,10.0,c,62.630001,4.999922,2024-12-30,1.000000,0.000000,0.000000,NaN,1.000000,1000.0,72.63,62.63,0.000001,itm
3,2024-12-23,2024-12-23,10.0,p,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,-0.000000,1000.0,72.63,0.00,0.010000,otm
4,2024-12-23,2024-12-23,20.0,c,52.630001,4.999922,2024-12-30,1.000000,0.000000,0.000000,NaN,1.000000,1000.0,72.63,52.63,0.000001,itm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555177,2024-10-17,2029-04-25,65.0,p,11.444299,0.241947,2029-04-30,-0.240271,0.008672,0.461183,-0.001094,-0.444279,1000.0,69.71,0.00,11.444299,otm
2555178,2024-10-17,2029-04-25,64.0,c,16.694535,0.243474,2029-04-30,0.768582,0.008441,0.451735,-0.006437,0.567961,1000.0,69.71,5.71,10.984535,itm
2555179,2024-10-17,2029-04-25,64.0,p,10.984536,0.243474,2029-04-30,-0.231418,0.008441,0.451735,-0.001134,-0.432039,1000.0,69.71,0.00,10.984536,otm
2555180,2024-10-17,2029-04-25,63.0,p,10.535058,0.245009,2029-04-30,-0.222714,0.008209,0.442075,-0.001169,-0.419672,1000.0,69.71,0.00,10.535058,otm


In [15]:
df_opt_chain = brn.chain.select_chain()
df_opt_chain

,datetime,expiration_date,strike,type,premium,iv,futures_expiration_date,delta,gamma,vega,theta,quick_delta,contract_size,futures_price,intrinsic_value,time_value,price_status
0,2024-12-23,2024-12-23,6.0,c,66.629997,4.999922,2024-12-30,1.0,0.0,0.0,NaN,1.0,1000.0,72.63,66.63,-0.000003,itm
1,2024-12-23,2024-12-23,6.0,p,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,-0.0,1000.0,72.63,0.00,0.010000,otm
2,2024-12-23,2024-12-23,10.0,c,62.630001,4.999922,2024-12-30,1.0,0.0,0.0,NaN,1.0,1000.0,72.63,62.63,0.000001,itm
3,2024-12-23,2024-12-23,10.0,p,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,-0.0,1000.0,72.63,0.00,0.010000,otm
4,2024-12-23,2024-12-23,20.0,c,52.630001,4.999922,2024-12-30,1.0,0.0,0.0,NaN,1.0,1000.0,72.63,52.63,0.000001,itm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,2024-12-23,2024-12-23,210.0,p,137.369995,4.999922,2024-12-30,-1.0,0.0,0.0,NaN,-1.0,1000.0,72.63,137.37,-0.000005,itm
352,2024-12-23,2024-12-23,225.0,c,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,0.0,1000.0,72.63,0.00,0.010000,otm
353,2024-12-23,2024-12-23,225.0,p,152.369995,4.999922,2024-12-30,-1.0,0.0,0.0,NaN,-1.0,1000.0,72.63,152.37,-0.000005,itm
354,2024-12-23,2024-12-23,250.0,c,0.010000,NaN,2024-12-30,NaN,NaN,NaN,NaN,0.0,1000.0,72.63,0.00,0.010000,otm


Futures types:
* Currency
* Index
* Commodities
* Stock
* Percent

Is crypto new type or currency?


BRN specification https://www.cmegroup.com/markets/energy/crude-oil/brent-crude-oil.html

Option chain https://www.barchart.com/futures/quotes/CB*0/options?futuresOptionsView=merged

In [16]:
atm_nearest_strikes = brn.chain.get_atm_nearest_strikes()
atm_strike = atm_nearest_strikes[0]
atm_strike

np.float64(72.75)

In [17]:
atm_strike_df = df_opt_chain[df_opt_chain['strike'] == atm_strike]
atm_strike_row = atm_strike_df[atm_strike_df['type']==OptionType.CALL.code].iloc[0]
atm_strike_df

,datetime,expiration_date,strike,type,premium,iv,futures_expiration_date,delta,gamma,vega,theta,quick_delta,contract_size,futures_price,intrinsic_value,time_value,price_status
144,2024-12-23,2024-12-23,72.75,c,0.53,4.999922,2024-12-30,0.0,0.0,0.0,NaN,0.0,1000.0,72.63,0.00,5.300000e-01,atm
145,2024-12-23,2024-12-23,72.75,p,0.12,4.999922,2024-12-30,-1.0,0.0,0.0,NaN,-1.0,1000.0,72.63,0.12,-2.682214e-09,atm


In [18]:
df_desk = brn.chain.get_desk(option_columns=['premium', 'iv' , 'delta', 'gamma', 'vega', 'theta', 'quick_delta', 'price_status', 'intrinsic_value', 'time_value', 'datetime', 'expiration_date', 'strike'])
res_col = ['premium_call', 'iv_call', 'delta_call', 'gamma_call', 'vega_call', 'theta_call', 'quick_delta_call', 'intrinsic_value_call', 'time_value_call', 'price_status_call', 'strike', 'price_status_put','premium_put', 'iv_put', 'delta_put', 'gamma_put', 'vega_put', 'theta_put', 'quick_delta_put', 'intrinsic_value_put', 'time_value_put', 'datetime', 'futures_price', 'expiration_date', 'futures_expiration_date']
df_desk[df_desk['strike'].isin(atm_nearest_strikes[:10])][res_col]

,premium_call,iv_call,delta_call,gamma_call,vega_call,theta_call,quick_delta_call,intrinsic_value_call,time_value_call,price_status_call,strike,price_status_put,premium_put,iv_put,delta_put,gamma_put,vega_put,theta_put,quick_delta_put,intrinsic_value_put,time_value_put,datetime,futures_price,expiration_date,futures_expiration_date
67,1.13,4.999922,1.0,0.0,0.0,NaN,1.0,1.13,-4.768367e-09,itm,71.50,otm,0.09,4.999922,0.0,0.0,0.0,NaN,-0.0,0.00,9.000000e-02,2024-12-23,72.63,2024-12-23,2024-12-30
68,0.88,4.999922,1.0,0.0,0.0,NaN,1.0,0.88,-4.768367e-09,itm,71.75,otm,0.11,4.999922,0.0,0.0,0.0,NaN,-0.0,0.00,1.100000e-01,2024-12-23,72.63,2024-12-23,2024-12-30
69,0.63,4.999922,1.0,0.0,0.0,NaN,1.0,0.63,-4.768367e-09,itm,72.00,otm,0.14,4.999922,0.0,0.0,0.0,NaN,-0.0,0.00,1.400000e-01,2024-12-23,72.63,2024-12-23,2024-12-30
70,0.38,4.999922,1.0,0.0,0.0,NaN,1.0,0.38,-4.768367e-09,itm,72.25,otm,0.19,4.999922,0.0,0.0,0.0,NaN,-0.0,0.00,1.900000e-01,2024-12-23,72.63,2024-12-23,2024-12-30
71,0.13,4.999922,1.0,0.0,0.0,NaN,1.0,0.13,-4.768367e-09,itm,72.50,otm,0.25,4.999922,0.0,0.0,0.0,NaN,-0.0,0.00,2.500000e-01,2024-12-23,72.63,2024-12-23,2024-12-30
72,0.53,4.999922,0.0,0.0,0.0,NaN,0.0,0.00,5.300000e-01,atm,72.75,atm,0.12,4.999922,-1.0,0.0,0.0,NaN,-1.0,0.12,-2.682214e-09,2024-12-23,72.63,2024-12-23,2024-12-30
73,0.39,4.999922,0.0,0.0,0.0,NaN,0.0,0.00,3.900000e-01,otm,73.00,itm,0.37,4.999922,-1.0,0.0,0.0,NaN,-1.0,0.37,4.768367e-09,2024-12-23,72.63,2024-12-23,2024-12-30
74,0.28,4.999922,0.0,0.0,0.0,NaN,0.0,0.00,2.800000e-01,otm,73.25,itm,0.62,4.999922,-1.0,0.0,0.0,NaN,-1.0,0.62,4.768367e-09,2024-12-23,72.63,2024-12-23,2024-12-30
75,0.21,4.999922,0.0,0.0,0.0,NaN,0.0,0.00,2.100000e-01,otm,73.50,itm,0.87,4.999922,-1.0,0.0,0.0,NaN,-1.0,0.87,4.768367e-09,2024-12-23,72.63,2024-12-23,2024-12-30
76,0.15,4.999922,0.0,0.0,0.0,NaN,0.0,0.00,1.500000e-01,otm,73.75,itm,1.12,4.999922,-1.0,0.0,0.0,NaN,-1.0,1.12,4.768367e-09,2024-12-23,72.63,2024-12-23,2024-12-30


The time value of the option decreases by the expiration date

ITM options have a low time value (low weight) in the total option price. In fact, the deeper an option is in the money, the lower its time value and the more it resembles a linear underlying asset (i.e., the corresponding futures).

OTM options have no intrinsic value (which is understandable – they are unprofitable to execute). The entire value of an out–of-money option is the intrinsic value. It is important to understand that if the market does not overcome the strike of this option, the final expiration option price will be zero.

ATM options have the maximum weight of the time value in the option price and the maximum non-linearity. For this reason, they are the most liquid on the market. When the underlying asset moves up or down from current levels, the liquidity of these options will decrease.

In [19]:
settlement_date, expiration_date = brn.chain.get_settlement_and_expiration_date()
settlement_date, expiration_date

(datetime.date(2024, 12, 23), datetime.date(2024, 12, 23))

In [23]:
brn.chart.init()
brn.chart.price.time_values()
brn.chart.show()

In [ ]:
brn.chart.price.time_values_for_strike(strike=atm_nearest_strikes[5])
brn.chart.price.time_values_for_distance(distance=0)
brn.chart.show()

In [20]:
df_time_value_strike = brn.analytic.price.time_value_series_by_strike_to_atm_distance(strike=atm_nearest_strikes[5])
df_time_value_strike.iloc[[0,int(len(df_time_value_strike)/2),-1]]

,datetime,strike,time_value
50,2024-01-02,72.5,8.551340e+00
13719,2024-06-13,79.0,4.938644e+00
34345,2024-12-23,72.0,-4.768367e-09


In [21]:
df_time_value_atm = brn.analytic.price.time_value_series_by_atm_distance(distance=0)
df_time_value_atm

,datetime,strike,time_value
27,2024-01-02,73.50,9.032935
120,2024-01-03,75.50,9.188117
237,2024-01-04,75.00,9.043473
414,2024-01-05,75.50,9.168615
455,2024-01-08,73.50,9.136093
...,...,...,...
33736,2024-12-17,73.25,0.754467
33916,2024-12-18,73.50,0.629862
34089,2024-12-19,73.00,0.534056
34269,2024-12-20,73.00,0.391472


In [24]:
brn.chart.init()
brn.chart.price.time_values([df_time_value_strike, df_time_value_atm], [f'Strike cur ATM {atm_strike}', 'ATM'])
brn.chart.show()

## Non-linearity of options

In [28]:
strikes = brn.chain.get_atm_nearest_strikes()
strike_start = sorted(filter(lambda x: x <= atm_strike * 0.95, strikes))[-1]
strike_end = sorted(filter(lambda x: x >= atm_strike * 1.1, strikes))[0]
strike_start, strike_end

(np.float64(69.0), np.float64(80.25))

In [29]:
fut_legs = [OptionLeg(strike=0, lots=1, type=LegType.FUTURE)]
fut_risk_profile, fut_risk_legs_pnl = brn.analytic.risk.chain_risk_profile(fut_legs)
fut_risk_profile = fut_risk_profile.loc[strike_start:strike_end]
fut_risk_profile.head(2)

,risk_pnl
strike,
69.00,-3.63
69.25,-3.38


In [30]:
call_legs = [OptionLeg(strike=atm_strike, lots=1, type=LegType.OPTION_CALL)]
call_risk_profile, call_risk_legs_pnl = brn.analytic.risk.chain_risk_profile(call_legs)
call_risk_profile=call_risk_profile.loc[strike_start:strike_end]
call_risk_profile.head(2)

,risk_pnl
strike,
69.00,-0.53
69.25,-0.53


In [31]:
call_legs_1 = [OptionLeg(strike=strikes[15], lots=1, type=LegType.OPTION_CALL)]
call_risk_profile_1, _ = brn.analytic.risk.chain_risk_profile(call_legs_1)
call_risk_profile_1=call_risk_profile_1.loc[strike_start:strike_end]
call_risk_profile_1.head(2)

,risk_pnl
strike,
69.00,-0.07
69.25,-0.07


In [28]:
data = [go.Scatter(x=fut_risk_profile.index.to_list(), y=fut_risk_profile['risk_pnl'].to_list(),mode='lines', name='fut'),
        go.Scatter(x=call_risk_profile.index.to_list(), y=call_risk_profile['risk_pnl'].to_list(),mode='lines', name=f'call atm {call_legs[0].strike}'),
        go.Scatter(x=call_risk_profile_1.index.to_list(), y=call_risk_profile_1['risk_pnl'].to_list(),mode='lines', name=f'call {call_legs_1[0].strike}')
       ] # markers, lines
iplot(data)